# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [9]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [10]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [11]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [12]:
ed = Website("https://platzi.com")
ed.links

['/',
 '/cursos/',
 '/precios/',
 '/business',
 '/2025/?utm_source=platzi_public_home',
 '/business/',
 'https://platzi.com/escuela/datos/',
 'https://platzi.com/escuela/ciberseguridad/',
 'https://platzi.com/escuela/liderazgo-management/',
 'https://platzi.com/escuela/ingles/',
 'https://platzi.com/escuela/web/',
 'https://platzi.com/escuela/marketing/',
 'https://platzi.com/escuela/producto/',
 'https://platzi.com/escuela/audiovisual/',
 'https://platzi.com/escuela/transformacion-digital-business/',
 'https://platzi.com/escuela/negocios/',
 'https://platzi.com/escuela/recursos-humanos/',
 'https://platzi.com/escuela/startups/',
 'https://platzi.com/escuela/finanzas/',
 'https://platzi.com/escuela/devops-cloud/',
 'https://platzi.com/escuela/diversidad/',
 'https://platzi.com/escuela/javascript/',
 'https://platzi.com/escuela/programacion-software/',
 'https://platzi.com/escuela/blockchain/',
 'https://platzi.com/escuela/diseno/',
 '#b2bPlans',
 'https://platzi.com/escuela/ingles/',
 

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [13]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [14]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [15]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [16]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://platzi.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/
/cursos/
/precios/
/business
/2025/?utm_source=platzi_public_home
/business/
https://platzi.com/escuela/datos/
https://platzi.com/escuela/ciberseguridad/
https://platzi.com/escuela/liderazgo-management/
https://platzi.com/escuela/ingles/
https://platzi.com/escuela/web/
https://platzi.com/escuela/marketing/
https://platzi.com/escuela/producto/
https://platzi.com/escuela/audiovisual/
https://platzi.com/escuela/transformacion-digital-business/
https://platzi.com/escuela/negocios/
https://platzi.com/escuela/recursos-humanos/
https://platzi.com/escuela/startups/
https://platzi.com/escuela/finanzas/
https://platzi.com/escuela/devops-cloud/
https://platzi.com/escuela/diversidad/
https://platzi.com/escuel

In [17]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [23]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://platzi.com")
huggingface.links

['/',
 '/cursos/',
 '/precios/',
 '/business',
 '/2025/?utm_source=platzi_public_home',
 '/business/',
 'https://platzi.com/escuela/datos/',
 'https://platzi.com/escuela/ciberseguridad/',
 'https://platzi.com/escuela/liderazgo-management/',
 'https://platzi.com/escuela/ingles/',
 'https://platzi.com/escuela/web/',
 'https://platzi.com/escuela/marketing/',
 'https://platzi.com/escuela/producto/',
 'https://platzi.com/escuela/audiovisual/',
 'https://platzi.com/escuela/transformacion-digital-business/',
 'https://platzi.com/escuela/negocios/',
 'https://platzi.com/escuela/recursos-humanos/',
 'https://platzi.com/escuela/startups/',
 'https://platzi.com/escuela/finanzas/',
 'https://platzi.com/escuela/devops-cloud/',
 'https://platzi.com/escuela/diversidad/',
 'https://platzi.com/escuela/javascript/',
 'https://platzi.com/escuela/programacion-software/',
 'https://platzi.com/escuela/blockchain/',
 'https://platzi.com/escuela/diseno/',
 '#b2bPlans',
 'https://platzi.com/escuela/ingles/',
 

In [24]:
get_links("https://platzi.com")

{'links': [{'type': 'homepage', 'url': 'https://platzi.com'},
  {'type': 'business page', 'url': 'https://platzi.com/business'},
  {'type': 'courses page', 'url': 'https://platzi.com/cursos/'},
  {'type': 'pricing page', 'url': 'https://platzi.com/precios/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [20]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [25]:
print(get_all_details("https://platzi.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://platzi.com'}, {'type': 'business page', 'url': 'https://platzi.com/business'}, {'type': 'courses page', 'url': 'https://platzi.com/cursos/'}]}
Landing page:
Webpage Title:
Platzi: Plataforma de aprendizaje profesional online
Webpage Contents:
Explorar
Comunidad
Planes
Empresas
Acceder
Aprovecha el precio especial
Ahorra:
$991
en Plan expert
Paga a
12
MSI
Termina en:
...
La escuela de tecnología
de Latinoamérica
Más de 5 millones de estudiantes y más de 3.000 empresas aprenden en Platzi
¿Qué quieres aprender?
Empieza cualquier curso sin costo.
O consigue
Platzi para tu empresa
Escuela de
Data Science e Inteligencia Artificial
Escuela de
Ciberseguridad
Escuela de
Liderazgo y Management
Escuela de
English Academy
Escuela de
Desarrollo Web
Escuela de
Marketing Digital
Escuela de
Producto
Escuela de
Contenido Audiovisual
Escuela de
Transformación Digital
Escuela de
Negocios
Escuela de
Talento y Recursos Humanos
Escuela de
Startu

In [33]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [34]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [28]:
get_brochure_user_prompt("Platzi", "https://platzi.com")

Found links: {'links': [{'type': 'home page', 'url': 'https://platzi.com'}, {'type': 'courses page', 'url': 'https://platzi.com/cursos/'}, {'type': 'pricing page', 'url': 'https://platzi.com/precios/'}, {'type': 'business page', 'url': 'https://platzi.com/business'}, {'type': 'data school', 'url': 'https://platzi.com/escuela/datos/'}, {'type': 'cybersecurity school', 'url': 'https://platzi.com/escuela/ciberseguridad/'}, {'type': 'leadership & management school', 'url': 'https://platzi.com/escuela/liderazgo-management/'}, {'type': 'English school', 'url': 'https://platzi.com/escuela/ingles/'}, {'type': 'web school', 'url': 'https://platzi.com/escuela/web/'}, {'type': 'marketing school', 'url': 'https://platzi.com/escuela/marketing/'}, {'type': 'product school', 'url': 'https://platzi.com/escuela/producto/'}, {'type': 'audiovisual school', 'url': 'https://platzi.com/escuela/audiovisual/'}, {'type': 'digital transformation & business school', 'url': 'https://platzi.com/escuela/transformac

'You are looking at a company called: Platzi\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nPlatzi: Plataforma de aprendizaje profesional online\nWebpage Contents:\nExplorar\nComunidad\nPlanes\nEmpresas\nAcceder\nAprovecha el precio especial\nAhorra:\n$991\nen Plan expert\nPaga a\n12\nMSI\nTermina en:\n...\nLa escuela de tecnología\nde Latinoamérica\nMás de 5 millones de estudiantes y más de 3.000 empresas aprenden en Platzi\n¿Qué quieres aprender?\nEmpieza cualquier curso sin costo.\nO consigue\nPlatzi para tu empresa\nEscuela de\nData Science e Inteligencia Artificial\nEscuela de\nCiberseguridad\nEscuela de\nLiderazgo y Management\nEscuela de\nEnglish Academy\nEscuela de\nDesarrollo Web\nEscuela de\nMarketing Digital\nEscuela de\nProducto\nEscuela de\nContenido Audiovisual\nEscuela de\nTransformación Digital\nEscuela de\nNegocios\nEscuela de\nTalento y Recu

In [29]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [30]:
create_brochure("Platzi", "https://platzi.com")

Found links: {'links': [{'type': 'homepage', 'url': 'https://platzi.com'}, {'type': 'business page', 'url': 'https://platzi.com/business'}]}


# Explore Platzi: Your Path to Professional Learning

## About Us
Platzi is the leading online learning platform in Latin America, dedicated to providing professionals with the tools they need to thrive in technology and innovation. With over **5 million students** and more than **3,000 companies** utilizing our courses, we are committed to enhancing skills through tailored learning experiences.

## What We Offer

### Comprehensive Learning Areas
We feature a wide array of specialized schools to cater to diverse professional growth:
- **Data Science and Artificial Intelligence**
- **Cybersecurity**
- **Leadership and Management**
- **English Academy**
- **Web Development**
- **Digital Marketing**
- **Product Management**
- **Audiovisual Content**
- **Business and Entrepreneurship**
- **Cloud Computing and DevOps**
- **Diversity and Inclusion**
- **Blockchain and Web3**
  
And many more! Each program is designed to take you from a novice to a professional through personalized learning pathways.

### Innovative Learning Experience
- **Free Courses:** Start any course without cost.
- **Interactive Progress Tests:** Continuously evaluate your learning.
- **Mobile Learning:** Download classes for offline study.
- **Certification Opportunities:** Obtain official certifications in English language proficiency (TOEFL), cloud technologies, and digital security (CompTIA).

## Company Culture
At Platzi, we foster a vibrant community passionate about technology and education. Our engaging learning environment encourages collaboration and innovation. We believe in continuous improvement and support our employees in their professional development journeys.

## Who Uses Platzi?
Our platform is a trusted resource for **students**, **professionals**, and **companies** looking to enhance their skills in alignment with market needs. By investing in their teams, organizations elevate their capabilities in critical fields like cybersecurity, AI, and more, ensuring they remain leaders in their respective industries.

## Careers at Platzi
Become part of a forward-thinking team that shapes the future of learning! We are always on the lookout for talented individuals who share our passion for education and technology. If you're ready to make an impact in the world of online learning, explore career opportunities with us.

## Join Us!
Whether you're an individual looking to expand your skillset, a company seeking comprehensive employee training, or a prospective recruit eager to join a vibrant culture — **Platzi is the place for you.**

**Discover your personalized learning path and start your journey with Platzi today!** [Visit our website](https://www.platzi.com) for more details.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [31]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [32]:
stream_brochure("Platzi", "https://platzi.com")

Found links: {'links': [{'type': 'home page', 'url': 'https://platzi.com/'}, {'type': 'courses page', 'url': 'https://platzi.com/cursos/'}, {'type': 'pricing page', 'url': 'https://platzi.com/precios/'}, {'type': 'business page', 'url': 'https://platzi.com/business'}, {'type': 'data school page', 'url': 'https://platzi.com/escuela/datos/'}, {'type': 'cybersecurity school page', 'url': 'https://platzi.com/escuela/ciberseguridad/'}, {'type': 'leadership management school page', 'url': 'https://platzi.com/escuela/liderazgo-management/'}, {'type': 'english school page', 'url': 'https://platzi.com/escuela/ingles/'}, {'type': 'web school page', 'url': 'https://platzi.com/escuela/web/'}, {'type': 'marketing school page', 'url': 'https://platzi.com/escuela/marketing/'}, {'type': 'product school page', 'url': 'https://platzi.com/escuela/producto/'}, {'type': 'audiovisual school page', 'url': 'https://platzi.com/escuela/audiovisual/'}, {'type': 'digital transformation business school page', 'url

# Platzi Brochure

---

## About Us

**Platzi** is the leading online professional learning platform in Latin America, committed to making education accessible to everyone. With over **5 million students** and **3,000 companies** utilizing our platform, we strive to empower individuals and teams through various tailored learning pathways.

## Our Mission

At Platzi, we believe that continuous education is key to professional growth in the fast-paced technology sector. Our platform provides a rich learning experience that helps students transition from beginners to professionals in their desired fields.

---

## Learning Opportunities

We offer a wide range of specialized schools covering various disciplines:

- **Data Science and AI**
- **Cybersecurity**
- **Leadership and Management**
- **English Academy**
- **Web Development**
- **Digital Marketing**
- **Product Management**
- **Audiovisual Content**
- **Digital Transformation**
- **Business Development**
- **Human Resources Talent**
- **Startups**
- **Finance and Investments**
- **DevOps and Cloud Computing**
- **Diversity and Inclusion**
- **JavaScript Programming**
- **Blockchain and Web3**
- **Graphic Design**

Choose your preferred learning path and start your journey **for free today!**

---

## Professional Growth

- **Customized Learning Paths**: Quickly generate a personalized route to enhance your skills.
- **Interactive Progress Tracking**: Measure your growth through hands-on assessments.
- **Community Support**: Engage with a vibrant community and get your questions answered.
- **Offline Learning**: Download classes for on-the-go learning without internet connectivity.

### Certification

Receive official certifications in:

- **English proficiency** through our partnership with ETS for TOEFL certification.
- **Cloud technologies** through our structured programs.
- **Digital security** with CompTIA certification.

---

## Company Culture

At Platzi, we prioritize fostering a **collaborative and inclusive work environment**. Our team of industry experts is dedicated to delivering high-quality content while supporting learners on their journey. We believe in empowering our employees and creating opportunities for professional development.

---

## Join Us

We are constantly looking for passionate individuals to join our team. By becoming a part of **Platzi**, you will play a vital role in shaping the future of education in Latin America.

### Current Opportunities

Explore exciting career paths with us in various roles and contribute to transforming how people learn and grow.

### Contact Us

Interested in learning more about our offerings or career opportunities? [Get in touch with us!](#)

---

### Learn. Grow. Succeed.

At Platzi, we pave the way for your professional education, enabling you to thrive in today’s technological landscape! 

--- 

## Follow Us

Stay connected and join our community to unlock endless possibilities! 

[Platzi Homepage](#) | [Explore Courses](#) | [Our Community](#)

---

In [35]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("Platzi", "https://platzi.com")

Found links: {'links': [{'type': 'home page', 'url': 'https://platzi.com/'}, {'type': 'business page', 'url': 'https://platzi.com/business'}, {'type': 'data school page', 'url': 'https://platzi.com/escuela/datos/'}, {'type': 'cybersecurity school page', 'url': 'https://platzi.com/escuela/ciberseguridad/'}, {'type': 'leadership and management school page', 'url': 'https://platzi.com/escuela/liderazgo-management/'}, {'type': 'english school page', 'url': 'https://platzi.com/escuela/ingles/'}, {'type': 'web school page', 'url': 'https://platzi.com/escuela/web/'}, {'type': 'marketing school page', 'url': 'https://platzi.com/escuela/marketing/'}, {'type': 'product school page', 'url': 'https://platzi.com/escuela/producto/'}, {'type': 'audiovisual school page', 'url': 'https://platzi.com/escuela/audiovisual/'}, {'type': 'digital transformation school page', 'url': 'https://platzi.com/escuela/transformacion-digital-business/'}, {'type': 'business school page', 'url': 'https://platzi.com/escue

# 🎉 Welcome to Platzi: Your Online Learning Playground! 🎉

## 📚 What is Platzi?
Platzi is the *ultimate* online learning platform turning students into professionals faster than you can say "Data Science"! With a vibrant community of over **5 million students** and **3,000 companies** hopping on board, we’re your one-stop shop for acquiring all those fancy skills needed to survive in the tech jungle.

---

## 🎓 Schools of Wonder!
At Platzi, we offer a treasure trove of courses! Have a seat in one of our **Schools** where you can find:

- 🥷 **Cybersecurity**: Because hackers don’t wear capes.
- 🤖 **Data Science & AI**: Let’s make data do the cha-cha!
- 🌐 **Web Development**: Build websites that even your grandma will think are cool.
- 🌟 **Digital Marketing**: Make your product the star of the internet!
- 🎨 **Graphic Design**: Turn your ideas into masterpieces (or at least something you won’t cringe at). 
- 🚀 **Startups**: Launch a business that can finally rival your neighbor’s taco truck.

And so much more! Check out our full course catalog and let your curiosity lead the way. 

---

## 💼 For Businesses
Hey companies! Tired of your employees Googling answers instead of actually learning? Let us help *elevate your talent*! Companies are using Platzi for training in everything from **leadership** to **innovation technologies**. 

### Why Choose Platzi?
- Tailored **learning paths** to turn your team from “Who’s that?” to “Wow, they made that!”
- **Live demos** and smart progress tracking—basically, like giving your team a health check-up but for their heads!
- Certification options for bragging rights, because who doesn’t love a digital trophy?

---

## 🌍 Join the Community
Join a community where you’re never alone (unless you choose to be—introverts are welcome too)! Our *interactive platform* keeps you connected, informed, and even slightly addicted to learning! Plus, with **downloadable classes**, you can let your brain feast on knowledge offline. Perfect for that slow Wi-Fi on vacation!

---

## 🎉 Careers at Platzi
Are you a tech-savvy wizard or just someone who loves mentoring aspiring geniuses? Platzi is on the lookout for new talents! If you dream of contributing to an educational revolution, putting your expertise to use, and maybe snagging a few ‘coolest workplace’ awards, then **Platzi** could be the perfect fit for you!

- Work with industry **experts**: Learn from the best, like our teachers who are *actually* making waves in the tech world.
- Flexible work culture: Because who doesn’t like wearing pajamas while learning?
- Growth opportunities that are as vast as the internet itself!

---

## 🤔 What Are You Waiting For?
Whether you’re a curious student, a forward-thinking company, or a potential recruit ready to ride the wave of online education, **Platzi** is your friendly neighborhood superhero in the realm of knowledge! 

**Join Us Today!** Start learning, training, and growing with us—because there’s always room for one more in our digital classroom! 

--- 

Let's get this learning party started! 🎉

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>